In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from treemort.utils.config import setup

config_file_path = "../configs/flair_unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/AerialImages"
conf.output_dir = os.path.join("..", conf.output_dir)

print(conf)

Namespace(data_folder='/Users/anisr/Documents/AerialImages', hdf5_file='AerialImageModel_ITD.h5', model='flair_unet', backbone=None, epochs=100, train_batch_size=8, val_batch_size=8, test_batch_size=8, train_crop_size=256, val_crop_size=256, test_crop_size=256, val_size=0.2, test_size=0.1, input_channels=4, output_channels=1, output_dir='../output/flair_unet', model_weights='best', learning_rate=0.0001, threshold=0.5, activation='tanh', loss='hybrid', resume=True)


In [3]:
import torch
import numpy as np
from skimage.measure import label, regionprops
from tqdm import tqdm

from treemort.training.output_processing import process_model_output

def calculate_geometric_properties(predictions):
    """
    Calculate basic geometric properties for each segmented dead tree.
    
    Args:
        predictions (torch.Tensor): A batch of binary segmentation masks of shape (N, H, W),
                                    where N is the batch size, H is height, and W is width.
                                    
    Returns:
        List of dictionaries containing geometric properties for each segmented object.
    """
    batch_properties = []
    
    # Convert tensor to numpy array for processing
    predictions = predictions.detach().cpu().numpy()
    
    for i in range(predictions.shape[0]):
        # Get binary mask for current prediction
        binary_mask = predictions[i].squeeze().astype(np.uint8)
        
        # Label connected components in the binary mask
        labeled_mask = label(binary_mask)
        
        # Extract properties for each labeled region (i.e., each dead tree)
        regions = regionprops(labeled_mask)
        image_properties = []
        
        for region in regions:
            properties = {
                'Area': region.area,
                'Perimeter': region.perimeter,
                'Centroid': region.centroid,
                'BoundingBox': region.bbox,
                'AspectRatio': region.bbox[2] / region.bbox[3] if region.bbox[3] != 0 else 0
            }
            image_properties.append(properties)
        
        batch_properties.append(image_properties)
    
    return batch_properties


def evaluate_geometric_properties(model, data_loader, conf, device, image_processor, ):
    """
    Evaluate geometric properties for the entire dataset using the trained model.
    
    Args:
        model: Trained segmentation model.
        data_loader: DataLoader for the dataset.
        conf: Configuration dictionary or object.
        device: Device to run the model on (e.g., 'cpu' or 'cuda').
        image_processor: Function or class to process model outputs.
    
    Returns:
        List of dictionaries containing geometric properties for all images in the dataset.
    """
    model.eval()
    all_geometric_properties = []
    all_geometric_properties_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating"):
            images = images.to(device)
            
            # Get model predictions
            predictions = process_model_output(model, images, conf, image_processor, labels, device=device)
            
            # Apply thresholding to get binary segmentation masks
            binary_masks = (predictions > 0.5).float()
            binary_labels = (labels > 0.5).float()
            
            # Calculate geometric properties
            geometric_properties = calculate_geometric_properties(binary_masks)
            all_geometric_properties.extend(geometric_properties)

            # Calculate geometric properties for labels
            geometric_properties = calculate_geometric_properties(binary_labels)
            all_geometric_properties_labels.extend(geometric_properties)
    
    return all_geometric_properties, all_geometric_properties_labels


In [4]:
from treemort.data.loader import prepare_datasets
from treemort.modeling.builder import resume_or_load

id2label = {0: "alive", 1: "dead"}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset, val_dataset, test_dataset, image_processor = prepare_datasets(conf)

model, optimizer, criterion, metrics, callbacks = resume_or_load(conf, id2label, len(train_dataset), device)    


/Users/anisr/Documents/TreeSeg/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Building model...
[INFO] FLAIR-UNet model configured with pre-trained weights.
[INFO] flair_unet model configured.
[INFO] Model successfully moved to cpu.
[INFO] Adam optimizer configured with learning rate 0.0001.
[INFO] Loss function 'hybrid' and associated metrics configured.
[INFO] Loaded weights from ../output/flair_unet/best.weights.pth.


In [5]:
from treemort.evaluation.evaluator import evaluator

evaluator(
            model,
            dataset=test_dataset,
            num_samples=len(test_dataset),
            batch_size=conf.test_batch_size,
            threshold=conf.threshold,
            model_name=conf.model,
            image_processor=image_processor,
        )

Evaluating: 100%|██████████| 76/76 [01:09<00:00,  1.10it/s, iterations_left=0] 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.5839793214884254e-09, 0.0, 9.250693793477618e-10, 1.0090817346023393e-09, 3.3670033556666553e-09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.3474178348652164e-09, 2.3474178348652164e-09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.019367990805945e-09, 5.282620176818478e-10, 0.0, 8.012820506399982e-10, 2.9761904673327665e-09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.3474178348652164e-09, 2.3474178348652164e-09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.6835016806675055e-09, 1.2315270920794002e-09, 3.891050568517313e-09, 0.0, 0.0, 2.5641024983563464e-08, 0.0, 0.0, 1.149425274144537e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0485829795603925e-09, 4.0485829795603925e-09, 0.0, 1.84

In [25]:
geometric_properties, geometric_properties_labels = evaluate_geometric_properties(model, test_dataset, conf, device, image_processor)

Evaluating: 100%|██████████| 76/76 [05:15<00:00,  4.15s/it]


In [31]:
for i, x in enumerate(geometric_properties_labels):
    if x != []:
        print(i, x)

45 [{'Area': 80.0, 'Perimeter': 32.242640687119284, 'Centroid': (211.375, 252.1375), 'BoundingBox': (206, 249, 218, 256), 'AspectRatio': 0.8515625}, {'Area': 307.0, 'Perimeter': 70.04163056034261, 'Centroid': (248.01954397394138, 239.62540716612378), 'BoundingBox': (238, 230, 256, 254), 'AspectRatio': 1.0078740157480315}]
47 [{'Area': 98.0, 'Perimeter': 34.72792206135785, 'Centroid': (187.4387755102041, 135.62244897959184), 'BoundingBox': (182, 131, 194, 141), 'AspectRatio': 1.375886524822695}, {'Area': 274.0, 'Perimeter': 76.38477631085024, 'Centroid': (206.8905109489051, 130.6897810218978), 'BoundingBox': (196, 121, 218, 141), 'AspectRatio': 1.5460992907801419}, {'Area': 146.0, 'Perimeter': 42.72792206135786, 'Centroid': (225.3972602739726, 142.23972602739727), 'BoundingBox': (219, 137, 233, 149), 'AspectRatio': 1.563758389261745}, {'Area': 256.0, 'Perimeter': 73.11269837220809, 'Centroid': (243.3203125, 152.8671875), 'BoundingBox': (231, 145, 256, 161), 'AspectRatio': 1.590062111801

In [36]:
id = 47
print(geometric_properties[id])
print(geometric_properties_labels[id])

[{'Area': 581.0, 'Perimeter': 112.5685424949238, 'Centroid': (201.2908777969019, 132.8382099827883), 'BoundingBox': (179, 119, 220, 144), 'AspectRatio': 1.5277777777777777}, {'Area': 7.0, 'Perimeter': 6.82842712474619, 'Centroid': (187.0, 124.28571428571429), 'BoundingBox': (186, 123, 189, 126), 'AspectRatio': 1.5}, {'Area': 176.0, 'Perimeter': 46.62741699796952, 'Centroid': (225.7215909090909, 141.76704545454547), 'BoundingBox': (219, 135, 234, 150), 'AspectRatio': 1.56}, {'Area': 226.0, 'Perimeter': 68.38477631085024, 'Centroid': (245.30088495575222, 152.00884955752213), 'BoundingBox': (233, 145, 256, 159), 'AspectRatio': 1.610062893081761}, {'Area': 242.0, 'Perimeter': 67.45584412271572, 'Centroid': (246.96280991735537, 111.14462809917356), 'BoundingBox': (237, 103, 256, 125), 'AspectRatio': 2.048}]
[{'Area': 98.0, 'Perimeter': 34.72792206135785, 'Centroid': (187.4387755102041, 135.62244897959184), 'BoundingBox': (182, 131, 194, 141), 'AspectRatio': 1.375886524822695}, {'Area': 274.

In [9]:
count = 0
for images, labels in train_dataset:
    for label in labels:
        if label.sum() > 0:
            count = count + 1

print(len(train_dataset)*8, count)

1056 500


In [7]:
t1 = 15367*conf.train_batch_size
t2 = 5567*conf.val_batch_size
t3 = 2426*conf.test_batch_size

print(t1, t2, t3)

t4 = 3606
t5 = t4 + t4*2
t6 = t4 + t4*7

print(t6, t5, t4)

print(t1+t2+t3, t4+t5+t6)

122936 44536 19408
28848 10818 3606
186880 43272


In [38]:
import numpy as np

def compare_properties(predicted, label):
    area_diff = abs(predicted['Area'] - label['Area'])
    perimeter_diff = abs(predicted['Perimeter'] - label['Perimeter'])
    aspect_ratio_diff = abs(predicted['AspectRatio'] - label['AspectRatio'])
    
    centroid_distance = np.linalg.norm(
        np.array(predicted['Centroid']) - np.array(label['Centroid'])
    )
    
    iou = calculate_iou(predicted['BoundingBox'], label['BoundingBox'])
    
    return {
        'Area Difference': area_diff,
        'Perimeter Difference': perimeter_diff,
        'Aspect Ratio Difference': aspect_ratio_diff,
        'Centroid Distance': centroid_distance,
        'IoU': iou
    }

def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# Example usage:
predicted_object = {'Area': 581.0, 'Perimeter': 112.5685424949238, 'Centroid': (201.2908777969019, 132.8382099827883), 'BoundingBox': (179, 119, 220, 144), 'AspectRatio': 1.5277777777777777}
label_object = {'Area': 98.0, 'Perimeter': 34.72792206135785, 'Centroid': (187.4387755102041, 135.62244897959184), 'BoundingBox': (182, 131, 194, 141), 'AspectRatio': 1.375886524822695}

comparison_results = compare_properties(predicted_object, label_object)
print(comparison_results)


{'Area Difference': 483.0, 'Perimeter Difference': 77.84062043356596, 'Aspect Ratio Difference': 0.15189125295508266, 'Centroid Distance': 14.129144508867478, 'IoU': 0.13095238095238096}


In [16]:
import streamlit as st
import geopandas as gpd

geojson_path = '/Users/anisr/Documents/TreeSeg/colab/geojson/M4124C_2017_1.geojson'

gdf = gpd.read_file(geojson_path)
gdf.geometry.centroid





0      POINT (341423.633 6801699.717)
1       POINT (341426.72 6801701.931)
2      POINT (341413.943 6801697.699)
3      POINT (341416.332 6801685.312)
4      POINT (341265.382 6801816.634)
                    ...              
115    POINT (341299.939 6801845.874)
116                       POINT EMPTY
117    POINT (341297.536 6801861.101)
118    POINT (341292.369 6801933.066)
119                       POINT EMPTY
Length: 120, dtype: geometry

In [11]:
geojson_data

'/Users/anisr/Documents/TreeSeg/colab/geojson/M4124C_2017_1.geojson'